### This is a script to run periodic model performance evaluation using the most recent 1 day of data

In [36]:
#### first get 2mon data and 1d dta. then join them. then reuse predict script.

In [32]:
import numpy as np
import pandas as pd
import os, time, warnings, random, shap, requests, optuna, datetime, joblib
import seaborn as sns
import matplotlib.pyplot as plt
import functools as ft
import yfinance as yf

from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split, KFold
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, f1_score, r2_score, mean_squared_error
from sklearn.inspection import permutation_importance
from xgboost import XGBClassifier, XGBRegressor

pd.set_option('display.max_columns', 100)
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.expand_frame_repr', False)
warnings.filterwarnings('ignore') 

time0 = time.time()

os.chdir('/home/jupyter/projects_gcp_cpu/spx/src')
os.getcwd()

'/home/jupyter/projects_gcp_cpu/spx/src'

In [28]:
df = pd.read_csv('gs://pmykola-streaming-projects/spg-stocks/data/data_last_2022125.csv')
df.Datetime = pd.to_datetime(df.Datetime)
df['time'] = df.Datetime.dt.time
df['date'] = df.Datetime.dt.date
df.dtypes

Datetime    datetime64[ns]
EEM                float64
EEMA               float64
EMXC               float64
Nasdaq             float64
Russel             float64
Spx                float64
VTHR               float64
time                object
date                object
dtype: object

In [29]:
df

,Datetime,EEM,EEMA,EMXC,Nasdaq,Russel,Spx,VTHR,time,date
0,2022-12-05 09:30:00,39.509998,65.650002,49.910000,11400.257812,1881.362305,4044.500000,181.544998,09:30:00,2022-12-05
1,2022-12-05 09:32:00,39.459999,65.529999,49.948898,11388.502930,1878.119385,4043.689941,NaN,09:32:00,2022-12-05
2,2022-12-05 09:34:00,39.500000,65.599998,NaN,11410.599609,1878.201294,4048.699951,NaN,09:34:00,2022-12-05
3,2022-12-05 09:36:00,39.540001,65.639999,NaN,11419.277344,1878.185059,4050.770020,NaN,09:36:00,2022-12-05
4,2022-12-05 09:38:00,39.459999,NaN,50.000000,11417.746094,1876.964966,4050.729980,NaN,09:38:00,2022-12-05
...,...,...,...,...,...,...,...,...,...,...
191,2022-12-05 15:52:00,39.084999,NaN,49.470001,11222.517578,1837.032227,3995.699951,NaN,15:52:00,2022-12-05
192,2022-12-05 15:54:00,39.084999,NaN,49.470001,11231.310547,1838.013672,3998.479980,179.253998,15:54:00,2022-12-05
193,2022-12-05 15:56:00,39.105000,64.910004,49.480000,11241.674805,1839.835571,4000.719971,179.289993,15:56:00,2022-12-05
194,2022-12-05 15:58:00,39.084999,65.019997,49.490002,11242.072266,1840.386230,3999.149902,179.500000,15:58:00,2022-12-05


In [ ]:
# get code from prediction, then modify it

In [30]:
### now i wanna do feature engineering for all assets 

asset_list = ['Spx', 'Nasdaq', 'Russel', 'EMXC', 'EEMA', 'EEM', 'VTHR']

for asset in asset_list:
    
    df[asset + '_ret'] = 100*(df[asset]/df[asset].shift(1)-1)
    df['s_' + asset + '_ret_1prd'] = (100*(df[asset]/df[asset].shift(1)-1)).shift(1)
    df['s_' + asset + '_ret_2prd'] = (100*(df[asset]/df[asset].shift(2)-1)).shift(1)
    df['s_' + asset + '_ret_4prd'] = (100*(df[asset]/df[asset].shift(4)-1)).shift(1)
    # display(df.shape, df.head(5))

    df.loc[df.time < datetime.time(9, 32, 0), 's_' + asset + '_1prd'] = np.nan
    df.loc[df.time < datetime.time(9, 33, 0), 's_' + asset + '_2prd'] = np.nan
    df.loc[df.time < datetime.time(9, 35, 0), 's_' + asset + '_4prd'] = np.nan

    dayopen.rename(columns={asset:asset+'_open'}, inplace=True)
    # dayopen.head()
    dayclose.rename(columns={asset:asset+'_close'}, inplace=True)
    dayclose_l1 = dayclose.copy()
    dayclose_l2 = dayclose.copy()
    dayclose_l1[asset+'_close_l1'] = dayclose_l1[asset+'_close'].shift(1)
    dayclose_l2[asset+'_close_l2'] = dayclose_l2[asset+'_close'].shift(2)

    # display(dayclose_l1.head(), dayclose_l2.head())

    df = pd.merge(df, dayopen[['date', asset + '_open']], on=['date'], how='left')
    df = pd.merge(df, dayclose_l1[['date', asset + '_close_l1']], on=['date'], how='left')
    df = pd.merge(df, dayclose_l2[['date', asset + '_close_l2']], on=['date'], how='left')

    df['s_' + asset + '_ret_open'] = (100*(df[asset]/df[asset + '_open']-1)).shift(1)
    df['s_' + asset + '_ret_close1'] = (100*(df[asset]/df[asset + '_close_l1']-1)).shift(1)
    df['s_' + asset + '_ret_close2'] = (100*(df[asset]/df[asset + '_close_l2']-1)).shift(1)

    cols_todrop = [x for x in list(df.columns) if asset in x and 'ret' not in x]
    df.drop(columns = cols_todrop, inplace=True)

display(time.time() - time0, df.tail())

425.13161611557007

,Datetime,time,date,Spx_ret,s_Spx_ret_1prd,s_Spx_ret_2prd,s_Spx_ret_4prd,s_Spx_ret_open,s_Spx_ret_close1,s_Spx_ret_close2,Nasdaq_ret,s_Nasdaq_ret_1prd,s_Nasdaq_ret_2prd,s_Nasdaq_ret_4prd,s_Nasdaq_ret_open,s_Nasdaq_ret_close1,s_Nasdaq_ret_close2,Russel_ret,s_Russel_ret_1prd,s_Russel_ret_2prd,s_Russel_ret_4prd,s_Russel_ret_open,s_Russel_ret_close1,s_Russel_ret_close2,EMXC_ret,s_EMXC_ret_1prd,s_EMXC_ret_2prd,s_EMXC_ret_4prd,s_EMXC_ret_open,s_EMXC_ret_close1,s_EMXC_ret_close2,EEMA_ret,s_EEMA_ret_1prd,s_EEMA_ret_2prd,s_EEMA_ret_4prd,s_EEMA_ret_open,s_EEMA_ret_close1,s_EEMA_ret_close2,EEM_ret,s_EEM_ret_1prd,s_EEM_ret_2prd,s_EEM_ret_4prd,s_EEM_ret_open,s_EEM_ret_close1,s_EEM_ret_close2,VTHR_ret,s_VTHR_ret_1prd,s_VTHR_ret_2prd,s_VTHR_ret_4prd,s_VTHR_ret_open,s_VTHR_ret_close1,s_VTHR_ret_close2
191,2022-12-05 15:52:00,15:52:00,2022-12-05,0.033048,0.029298,-0.016524,0.072150,NaN,NaN,NaN,0.043458,0.041229,-0.015537,0.075081,NaN,NaN,NaN,0.070932,-0.009980,-0.024119,-0.042772,NaN,NaN,NaN,0.020223,0.000000,0.000000,-0.010110,NaN,NaN,NaN,NaN,-0.030823,-0.015414,NaN,NaN,NaN,NaN,0.012787,0.002821,-0.012786,0.025601,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192,2022-12-05 15:54:00,15:54:00,2022-12-05,0.069576,0.033048,0.062356,0.108731,NaN,NaN,NaN,0.078351,0.043458,0.084706,0.120997,NaN,NaN,NaN,0.053426,0.070932,0.060945,0.072776,NaN,NaN,NaN,0.000000,0.020223,0.020223,0.060685,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.012787,0.015609,0.051198,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193,2022-12-05 15:56:00,15:56:00,2022-12-05,0.056021,0.069576,0.102647,0.086105,NaN,NaN,NaN,0.092280,0.078351,0.121844,0.106288,NaN,NaN,NaN,0.099123,0.053426,0.124396,0.100246,NaN,NaN,NaN,0.020211,0.000000,0.020223,0.020223,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.051172,0.000000,0.012787,0.000000,NaN,NaN,NaN,0.020081,NaN,NaN,0.058054,NaN,NaN,NaN
194,2022-12-05 15:58:00,15:58:00,2022-12-05,-0.039245,0.056021,0.125636,0.188069,NaN,NaN,NaN,0.003536,0.092280,0.170703,0.255554,NaN,NaN,NaN,0.029930,0.099123,0.152602,0.213640,NaN,NaN,NaN,0.020215,0.020211,0.020211,0.040438,NaN,NaN,NaN,0.169455,NaN,NaN,0.015412,NaN,NaN,NaN,-0.051146,0.051172,0.051172,0.066788,NaN,NaN,NaN,0.117132,0.020081,NaN,NaN,NaN,NaN,NaN
195,2022-12-05 16:00:00,16:00:00,2022-12-05,-0.007747,-0.039245,0.016754,0.119418,NaN,NaN,NaN,-0.018989,0.003536,0.095819,0.217779,NaN,NaN,NaN,-0.008921,0.029930,0.129083,0.253639,NaN,NaN,NaN,0.000000,0.020215,0.040429,0.060660,NaN,NaN,NaN,-0.169168,0.169455,NaN,0.215782,NaN,NaN,NaN,0.012795,-0.051146,0.000000,0.012787,NaN,NaN,NaN,NaN,0.117132,0.137237,NaN,NaN,NaN,NaN


In [34]:
### do prediction ###

model_path = '/home/jupyter/project_repos/spg_stocks/spg_stocks/stocks-app/en_model.pkl'
trained_model = joblib.load(open(model_path, "rb"))

X = df.tail(1)
X.drop(columns = ['Datetime',
                  'time', 
                  'date', 
                  'Spx_ret', 
                  'Nasdaq_ret', 
                  'Russel_ret', 
                  'EEMA_ret', 
                  'EEM_ret', 
                  'EMXC_ret', 
                  'VXUS_ret', 
                  'VTHR_ret'], 
                  inplace=True,
                  errors = 'ignore')

if(X.count().sum() < X.shape[1]):
    print(f'''There are {X.shape[1] - X.count().sum()} missing values. 
          There will be an error''')

print(trained_model.predict(X))

print('Total time: ', time.time()-time0)

There are 23 missing values. 
          There will be an error


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
# evaluate performance

In [35]:
X

,s_Spx_ret_1prd,s_Spx_ret_2prd,s_Spx_ret_4prd,s_Spx_ret_open,s_Spx_ret_close1,s_Spx_ret_close2,s_Nasdaq_ret_1prd,s_Nasdaq_ret_2prd,s_Nasdaq_ret_4prd,s_Nasdaq_ret_open,s_Nasdaq_ret_close1,s_Nasdaq_ret_close2,s_Russel_ret_1prd,s_Russel_ret_2prd,s_Russel_ret_4prd,s_Russel_ret_open,s_Russel_ret_close1,s_Russel_ret_close2,s_EMXC_ret_1prd,s_EMXC_ret_2prd,s_EMXC_ret_4prd,s_EMXC_ret_open,s_EMXC_ret_close1,s_EMXC_ret_close2,s_EEMA_ret_1prd,s_EEMA_ret_2prd,s_EEMA_ret_4prd,s_EEMA_ret_open,s_EEMA_ret_close1,s_EEMA_ret_close2,s_EEM_ret_1prd,s_EEM_ret_2prd,s_EEM_ret_4prd,s_EEM_ret_open,s_EEM_ret_close1,s_EEM_ret_close2,s_VTHR_ret_1prd,s_VTHR_ret_2prd,s_VTHR_ret_4prd,s_VTHR_ret_open,s_VTHR_ret_close1,s_VTHR_ret_close2
195,-0.039245,0.016754,0.119418,NaN,NaN,NaN,0.003536,0.095819,0.217779,NaN,NaN,NaN,0.02993,0.129083,0.253639,NaN,NaN,NaN,0.020215,0.040429,0.06066,NaN,NaN,NaN,0.169455,NaN,0.215782,NaN,NaN,NaN,-0.051146,0.0,0.012787,NaN,NaN,NaN,0.117132,0.137237,NaN,NaN,NaN,NaN
